In [1]:
import time
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
def init_browser():
    executable_path = {"executable_path": 'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
def scrape():
    # initialize browser
    browser = init_browser()
    
    #create blank dictionary for returns
    mars_news = {}
    
    # set the url and visit
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    mars_news["news_title"] = soup.find("div", class_="content_title").next_element.get_text()
    mars_news["news_p"] = soup.find("div", class_="article_teaser_body").get_text()
    
    return mars_news

mars = scrape()
print(mars["news_title"])
print(mars["news_p"])

Mars Mission Sheds Light on Habitability of Distant Planets
How long might a rocky, Mars-like planet be habitable if it were orbiting a red dwarf star?


In [5]:
def scrape_img():
    browser = init_browser()
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')

    image = soup.find('a', class_='button fancybox')
    featured_image_url = "https://www.jpl.nasa.gov" + image["data-fancybox-href"]
    
    return featured_image_url

featured_image_url = scrape_img()
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA08097_ip.jpg


In [35]:
def weather_scrape():
    browser = init_browser()
    url = "https://twitter.com/marswxreport?lang=en"
    
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    
    weather = soup.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
    
    for i in weather:
        j = i.text.split()
        if j[0] != "Sol":
            continue
        else:
            mars_weather = i.text
            break

    
        
    return mars_weather

mars_weather = weather_scrape()
print(mars_weather)

Sol 1917 (Dec 27, 2017), Sunny, high -19C/-2F, low -80C/-112F, pressure at 7.86 hPa, daylight 05:48-17:31


In [11]:
def fact_scrape():
    url = "https://space-facts.com/mars/"
    
    mars_table = pd.read_html(url)
    
    return mars_table

table = fact_scrape()

mars_fact_df = table[0]
mars_fact_df.columns = ["Characteristic", "Measurement"]

mars_fact_df.head()
    

,Characteristic,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [32]:

browser = init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_url = "https://astrogeology.usgs.gov"

browser.visit(url)
time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'lxml')

url_list = []
title_list =[]
jpg_url_list =[]
hemisphere_image_urls = []

results = soup.find('div', class_="collapsible results")
links = results.find_all('a', class_="item product-item")
titles= results.find_all('h3')

for link in links:
    url_list.append(base_url+link["href"])
    
for title in titles:
    title_list.append(title.text)
    
for item in url_list:
    browser.visit(item)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    
    downloads = soup.find('div', class_="downloads")
    jpg = downloads.find_all('a')
    jpg_url = jpg[0]["href"]
    jpg_url_list.append(jpg_url)
    

for i in range(0,len(title_list)):
    hemisphere_image_urls.append({"title": title_list[i], "img_url": jpg_url_list[i]})

In [39]:
scraped_data = {}

scraped_data["news"] = mars
scraped_data["feat_img"] = featured_image_url
scraped_data["weather"] = mars_weather
scraped_data["facts"] = table
scraped_data["hemispheres"] = hemisphere_image_urls

In [40]:
scraped_data

{'facts': [         Characteristic                    Measurement
  0  Equatorial Diameter:                       6,792 km
  1       Polar Diameter:                       6,752 km
  2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
  3                Moons:            2 (Phobos & Deimos)
  4       Orbit Distance:       227,943,824 km (1.52 AU)
  5         Orbit Period:           687 days (1.9 years)
  6  Surface Temperature:                  -153 to 20 °C
  7         First Record:              2nd millennium BC
  8          Recorded By:           Egyptian astronomers],
 'feat_img': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA08097_ip.jpg',
 'hemispheres': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
 